In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Este notebook require Tensorflow 2.X

In [2]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()
    
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)
max_id = len(tokenizer.word_index) 

In [3]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [4]:
model = keras.models.load_model('shakespeare-stateless.h5')
#model = keras.models.load_model('shakespeare-stateful.h5')

In [5]:
X_new = preprocess(["How are yo"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # primera oracion ultimo caracter.

'u'

In [6]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

La temperatura es un hiperparámetro de los LSTM (y las redes neuronales en general) que se utiliza para controlar la aleatoriedad de las predicciones escalando los logits antes de aplicar softmax.

Cuando la temperatura es 1, calculamos el softmax directamente en los logits (la salida sin escala de capas anteriores)

https://cs.stackexchange.com/questions/79241/what-is-temperature-in-lstm-and-neural-networks-generally

In [7]:
tf.random.set_seed(42)

next_char("How are yo", temperature=1)

'u'

In [8]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [10]:
print(complete_text("the", temperature=1))

they course:
i cannot be the sorrow some tribures.
yo


In [11]:
print(complete_text("the", temperature=2))

thee:
bmevely!--my?glaz high beail! deai. ruli-yourma


In [12]:
print(complete_text("the", temperature=0.6))

ther, have men the death,
she hath care to be the ten


In [13]:
print(complete_text("the", temperature=0.2))

ther the country with the soul
the straight the poor 
